In [18]:
[[[u, v] for v in range(3)] for u in range(10)]

[[[0, 0], [0, 1], [0, 2]],
 [[1, 0], [1, 1], [1, 2]],
 [[2, 0], [2, 1], [2, 2]],
 [[3, 0], [3, 1], [3, 2]],
 [[4, 0], [4, 1], [4, 2]],
 [[5, 0], [5, 1], [5, 2]],
 [[6, 0], [6, 1], [6, 2]],
 [[7, 0], [7, 1], [7, 2]],
 [[8, 0], [8, 1], [8, 2]],
 [[9, 0], [9, 1], [9, 2]]]

In [15]:
import numpy as np
import math

# latitude 纬度（平行于赤道）
PI = np.pi
TAU = 2*PI
n_lat_lines = 10
theta_step = PI / n_lat_lines

theta = np.arange(0, PI, theta_step)
print(theta)

print("\n")
theta = theta[2]
phi = np.linspace(0, TAU, int(2 * n_lat_lines * math.sin(theta)) + 1)
print(phi)

[0.         0.31415927 0.62831853 0.9424778  1.25663706 1.57079633
 1.88495559 2.19911486 2.51327412 2.82743339]


[0.         0.57119866 1.14239733 1.71359599 2.28479466 2.85599332
 3.42719199 3.99839065 4.56958931 5.14078798 5.71198664 6.28318531]


In [6]:
# 列表推导式，双层循环
# 先对内层循环
coordinates = [(x, y) for x in range(3) for y in range(x+1)]
print(coordinates)

[(0, 0), (1, 0), (1, 1), (2, 0), (2, 1), (2, 2)]


In [12]:
import numpy as np

n_lat_lines = 18
theta_step = 180 / n_lat_lines

for theta in np.arange(0, 180, theta_step):
    print(theta)

0.0
10.0
20.0
30.0
40.0
50.0
60.0
70.0
80.0
90.0
100.0
110.0
120.0
130.0
140.0
150.0
160.0
170.0


In [ ]:
import numpy as np

n_lat_lines = 18
theta_step = 180 / n_lat_lines

for theta in np.arange(0, 180, theta_step):
    print(theta + theta_step * (phi / TAU))


In [16]:
import numpy as np

for phi in np.linspace(0, 360, 100):
    print(phi)

0.0
3.6363636363636362
7.2727272727272725
10.909090909090908
14.545454545454545
18.18181818181818
21.818181818181817
25.454545454545453
29.09090909090909
32.72727272727273
36.36363636363636
40.0
43.63636363636363
47.27272727272727
50.90909090909091
54.54545454545455
58.18181818181818
61.81818181818181
65.45454545454545
69.0909090909091
72.72727272727272
76.36363636363636
80.0
83.63636363636364
87.27272727272727
90.9090909090909
94.54545454545455
98.18181818181817
101.81818181818181
105.45454545454545
109.0909090909091
112.72727272727272
116.36363636363636
120.0
123.63636363636363
127.27272727272727
130.9090909090909
134.54545454545453
138.1818181818182
141.8181818181818
145.45454545454544
149.0909090909091
152.72727272727272
156.36363636363635
160.0
163.63636363636363
167.27272727272728
170.9090909090909
174.54545454545453
178.1818181818182
181.8181818181818
185.45454545454544
189.0909090909091
192.72727272727272
196.36363636363635
200.0
203.63636363636363
207.27272727272725
210.909090

In [2]:
import numpy as np

def project_to_xy_plane(p1, p2):
    """
    如何理解这个函数？
    函数目的: 求出从p1到p2的直线与xy平面的交点

    为了方便讨论, 假设p1和p2在xoy平面的上方, 且p1在p2之上
    vect是一条从p1指向p2的向量
    (z2 / vect[2]) * vect也就是
    (z1 / (z2-z1)) * vect, 就是把vect向量延长z1/(z1-z2)倍, 且取反方向
    从几何角度来看, 延长后的向量的起点(因为取反)恰好在xoy平面上
    假设起点为p3
    那么延长后的向量(z1 / (z1-z2)) * vect = p1 - p3
    进而: p3 = p1 - (p1 - p3)
    """
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    if z2 < z1:
        z2 = z1 + 1e-2  # TODO, bad hack
    vect = p2 - p1
    return p1 - (z2 / vect[2]) * vect
    

p1 = np.array([1, 1, 3])
p2 = np.array([1, 1, 2])
project_to_xy_plane(p1, p2)

array([ 1.  ,  1.  , -0.01])

In [4]:
import numpy as np

def project_to_xy_plane(p1, p2):
    """
    如何理解这个函数？
    函数目的: 求出从p1到p2的直线与xy平面的交点

    为了方便讨论, 假设p1和p2在xoy平面的上方, 且p1在p2之上
    vect是一条从p1指向p2的向量
    (z2 / vect[2]) * vect也就是
    (z1 / (z2-z1)) * vect, 就是把vect向量延长z1/(z1-z2)倍, 且取反方向
    从几何角度来看, 延长后的向量的起点(因为取反)恰好在xoy平面上
    假设起点为p3
    那么延长后的向量(z1 / (z1-z2)) * vect = p1 - p3
    进而: p3 = p1 - (p1 - p3)
    """
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    vect = p2 - p1
    return p1 - (z1 / vect[2]) * vect
    
p1 = np.array([1, 1, 3])
p2 = np.array([1, 1, 2])
project_to_xy_plane(p1, p2)

array([1., 1., 0.])

In [5]:
import numpy as np

data = np.array([[1, 2, 3],
                 [4, 5, 6],
                 [7, 8, 9]])

def custom_function(arr):
    return np.sum(arr)

result_axis_0 = np.apply_along_axis(custom_function, axis=0, arr=data)
result_axis_1 = np.apply_along_axis(custom_function, axis=1, arr=data)

print("Result along axis 0:", result_axis_0)
print("Result along axis 1:", result_axis_1)


Result along axis 0: [12 15 18]
Result along axis 1: [ 6 15 24]


In [28]:
import numpy as np

# Define u_range and v_range
u_range = range(0, 10, 1)
v_range = range(0, 10, 1)

# Use nested list comprehension and conditional statement to generate the result
result = [[[u, v] for v in v_range] for u in u_range]
for i in range(len(result)):
    for j in range(len(result[i])):
        u = result[i][j][0]
        v = result[i][j][1] 
        if u + v < 10:
            result[i][j]=[0,0]

# Print the result
for row in result:
    print(row)


[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [1, 9]]
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [2, 8], [2, 9]]
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [3, 7], [3, 8], [3, 9]]
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [4, 6], [4, 7], [4, 8], [4, 9]]
[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [5, 5], [5, 6], [5, 7], [5, 8], [5, 9]]
[[0, 0], [0, 0], [0, 0], [0, 0], [6, 4], [6, 5], [6, 6], [6, 7], [6, 8], [6, 9]]
[[0, 0], [0, 0], [0, 0], [7, 3], [7, 4], [7, 5], [7, 6], [7, 7], [7, 8], [7, 9]]
[[0, 0], [0, 0], [8, 2], [8, 3], [8, 4], [8, 5], [8, 6], [8, 7], [8, 8], [8, 9]]
[[0, 0], [9, 1], [9, 2], [9, 3], [9, 4], [9, 5], [9, 6], [9, 7], [9, 8], [9, 9]]


In [25]:
import numpy as np

# Define u_range and v_range
u_range = range(0, 10, 1)
v_range = range(0, 10, 1)

# Use nested list comprehension and conditional statement to generate the result
result = [[[0, 0] for v in v_range] for u in u_range]
result = [[[u, v] for v in v_range if u + v < 10 ] for u in u_range]

# Convert the result to a regular Python list
result = list(result)

# Print the result
for row in result:
    print(row)


[[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9]]
[[1, 0], [1, 1], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8]]
[[2, 0], [2, 1], [2, 2], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7]]
[[3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6]]
[[4, 0], [4, 1], [4, 2], [4, 3], [4, 4], [4, 5]]
[[5, 0], [5, 1], [5, 2], [5, 3], [5, 4]]
[[6, 0], [6, 1], [6, 2], [6, 3]]
[[7, 0], [7, 1], [7, 2]]
[[8, 0], [8, 1]]
[[9, 0]]


In [11]:
u_range = list(range(0, 10, 1))
u_range

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]